In [1]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
#from bokeh.plotting import figure, show, output_file


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'Label_synergy4000.xlsx')

In [2]:
# 合并drugA和drugB为一列作为索引
temp1 = []
index_l = []
for index,row in data2.iterrows():
    ct = np.array(data1.loc[row["DrugA"],:]) + np.array(data1.loc[row["DrugB"],:])
    tp = []
    for i in ct:
        if i >= 1:
            tp.append(1)
        else:
            tp.append(0)
    temp1.append(tp)
    index_l.append((row["DrugA"],row["DrugB"]))

In [3]:
# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]        
        b=q[i]
        if (a==1)|(b==1):
            tep1=tep1+1
        if (a==1)&(b==1):
            tep2=tep2+1 
    c=float(tep2 / tep1)        
    return c

In [4]:
# 相互之间做一次tanimoto
temp=[]  
for i in temp1:
    tmp=[]   
    a=np.array(i) #取出一组值
    for j in temp1:
        b=np.array(j)    #取出一组值
        tp=tanimoto(a,b)
        tmp.append(tp)
    temp.append(tmp)
Tanimoto=pd.DataFrame(temp,columns=index_l,index=index_l)

In [5]:
# 获取栏目数组
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1])
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp))
columns = data2.columns

In [6]:
# 删去对应行后的字典
dict_lines = {}
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct])
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y
    ct += 1

In [7]:
coum=5

In [8]:
# 分离x y数据构造字典
dict_columns = {}
for i in columns: 
    
    # b
    y = data2.loc[:,i]
    
    # A
    temp = []
    for j in columns:
        if j != i:
            temp.append(j)
    m = data2.loc[:, temp[0]]
    tp = [m]
    for j in temp[1:]:
        tp.append(data2.loc[:, j])
    x = pd.DataFrame(tp).T
    

    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    max_l = df.index[-coum:]
    tp = []
    for j in max_l:
        tp.append(data2.loc[:, j])
    max_5x = pd.DataFrame(tp).T
    
    # 存储
    data = {
        "x": x,
        "y": y,
        "max_relevance": max_l,
        "max_5x": max_5x
    }
    dict_columns[i] = data

In [9]:
len(dict_columns)

583

In [10]:
dict_columns_split = {}
for i in dict_columns: # 第几列
    count = 0 # 第几行 
    for j in list(dict_columns[i]['y'].index):
        dict_columns_split[f'{i},{j}']={}
        dict_columns_split[f'{i},{j}']['y_line']=dict_columns[i]["y"][count]
        dict_columns_split[f'{i},{j}']['max_x_line']=dict_columns[i]["max_5x"].iloc[count,:].values
        count += 1
# 第几个 第几行
len(dict_columns_split)

20405

In [11]:
# 删去对应行后的字典
dict_high_lines = {}
count = 0
cct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,cct])
    y.pop(count)
    count += 1
    dict_high_lines[str(i)[1:-1]] = y

    tp = []
    for j in columns:
        if str(i)[1:-1] != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[str(i)[1:-1]], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)

    max_l = df.index[-coum:] 
    tp = []
    for j in max_l:
        tp.append(Tanimoto.iloc[list(columns).index(j), :])
    max_5y = list(pd.DataFrame(tp).T.iloc[[list(columns).index(str(i)[1:-1])]].values)[0]

    data = {
        "y": y,
        "max_relevance": max_l,
        "high_5": max_5y
    }
    dict_high_lines[str(i)[1:-1]] = data
    cct += 1

In [12]:
# 筛选每个药物对前5药物对得分平均值大于0.90的药物对
end_score = []
name_dict_high_lines = []
for elem in dict_high_lines:
    end_score.append(dict_high_lines[elem]['high_5'].mean())
    name_dict_high_lines.append(elem)
dict_high_lines_temp = {}
for score in end_score:
    if score >= 0.90:
        dict_high_lines_temp[name_dict_high_lines[end_score.index(score)]] = dict_high_lines[name_dict_high_lines[end_score.index(score)]]
print(len(dict_high_lines_temp))
dict_high_lines = dict_high_lines_temp.copy()

56


In [13]:
dict_high_lines_split = {}
for i in dict_high_lines:
    count = 0
    for j in dict_high_lines[i]['max_relevance']:
        dict_high_lines_split[f'{i},{j}']=dict_high_lines[i]['high_5'][count]
        count += 1
# 相似性
dict_high_lines_split

{"'VINORELBINE', 'SUNITINIB','MK-4541', 'VINORELBINE'": 0.8850574712643678,
 "'VINORELBINE', 'SUNITINIB','L778123', 'VINORELBINE'": 0.8953488372093024,
 "'VINORELBINE', 'SUNITINIB','VINORELBINE', 'SORAFENIB'": 0.896551724137931,
 "'VINORELBINE', 'SUNITINIB','DOXORUBICIN', 'SUNITINIB'": 0.896551724137931,
 "'VINORELBINE', 'SUNITINIB','VINBLASTINE', 'SUNITINIB'": 0.963855421686747,
 "'AZD1775', 'MK-4827','METHOTREXATE', 'AZD1775'": 0.9036144578313253,
 "'AZD1775', 'MK-4827','AZD1775', 'BEZ-235'": 0.9102564102564102,
 "'AZD1775', 'MK-4827','AZD1775', 'MK-2206'": 0.9367088607594937,
 "'AZD1775', 'MK-4827','AZD1775', 'MK-8776'": 0.9382716049382716,
 "'AZD1775', 'MK-4827','AZD1775', 'ABT-888'": 0.9743589743589743,
 "'VINORELBINE', 'DASATINIB','DEXAMETHASONE', 'DASATINIB'": 0.8947368421052632,
 "'VINORELBINE', 'DASATINIB','DASATINIB', 'GELDANAMYCIN'": 0.8958333333333334,
 "'VINORELBINE', 'DASATINIB','PACLITAXEL', 'DASATINIB'": 0.8969072164948454,
 "'VINORELBINE', 'DASATINIB','VINORELBINE', 'M

In [27]:
import threading
import queue
from tqdm import tqdm

q = queue.Queue()
result_q = queue.Queue()

for elem in range(35):
    q.put(elem)

def temp_f():
    while True:
        if q.qsize() != 0:
            elem = q.get()
            mse_split = []
            lin = np.linspace(0.01, 0.1, 50)
            for alp in lin:
                r1 = []
                ct = 0
                for linename in dict_high_lines:
                    count = 0
                    pp = []
                    for i in dict_high_lines_split:
                        if count >= 5 * ct and count < (5 * ct + 5):
                            pp.append(np.exp(-pow(1 - dict_high_lines_split[i], 2) / (2 * pow(alp, 2))))
                        count += 1
                    ct += 1
                    r1.append(np.sum(pp * dict_columns_split[f"{linename},{elem}"]['max_x_line']) -
                              dict_columns_split[f"{linename},{elem}"]['y_line'])
                mse_split.append(np.mean(np.square(r1)))
            mse = np.min(mse_split)
            alp_best = lin[mse_split.index(np.min(mse_split))]
            result_q.put((mse, alp_best))
        else:
            break


threads = []
for i in tqdm(range(35)):
    t = threading.Thread(target=temp_f)
    threads.append(t)
    t.start()

print('主程序运行中...')

# 等待所有线程任务结束。
for t in threads:
    t.join()

print("所有线程任务完成")

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:03<00:00,  8.77it/s]


主程序运行中...
所有线程任务完成


In [28]:
result_mse = []
result_alp = []
for i in range(result_q.qsize()):
    j = result_q.get()
    result_mse.append(j[0])
    result_alp.append(j[1])

In [29]:
np.sum(result_mse)/38

335.27774758656824

In [31]:
result_alp

35

import pickle

with open('药物对alpha.pkl', 'wb') as file:
    pickle.dump(result_alp, file) # 权重

In [ ]:
def dictW(alpha):
    # 运算构造参数W字典 
    dict_W = {}
    for elem in dict_high_lines:
        r = np.array(dict_high_lines[elem]["high_5"])
        W1= pow((1-r),2)
        W = np.exp(-W1/(2*pow(alpha,2)))
        dict_W[elem] = W
    return dict_W

#计算MSE
def dictr(dict_W):
# 残差字典 
    # r = max(|A×W - b|)
    dict_r = {}
    dict_r_list = {}
    for elem,w in zip(dict_columns, dict_W):
        A = np.array(dict_columns[elem]["max_5x"].values) # x----->max_5x
        b = np.array(dict_columns[elem]["y"].values)
        W = dict_W[w]
        temp1 = np.dot(A,W) - b
        temp2 = np.square(temp1)
        temp3 = np.mean(temp2)
        dict_r[w] = temp3
    return dict_r

def dirlist(dict_r):
    #计算RMSE  
    dir_list = []
    for elem in dict_r:
        dir_list.append(np.sqrt(dict_r[elem]));
    return dir_list

In [ ]:
dev=100
lin=np.linspace(0.001,1,1000)
for alpha in lin:
    
    dict_W = dictW(alpha)
    
    dict_r = dictr(dict_W)

    dir_list=dirlist(dict_r)
    
    temp=np.mean(dir_list)
                      
    #误差判断
    if(temp>=dev):
        continue
    else:
        dev=temp
        al=alpha

alpha=al
dict_W = dictW(alpha)
dict_r= dictr(dict_W)

In [ ]:
# MSE
np.array(list(dict_r.values())).mean()

# 存储最终数据
import pickle


with open('药物对.pkl', 'wb') as file:
    pickle.dump(dict_W, file) # 权重
    pickle.dump(dict_columns, file) # 药物对得分所有数据
    pickle.dump(dict_high_lines, file) # 药物对得分高的数据

In [ ]:
#3 477.4773065950993
#4 475.1346138628828
#5 475.17855530380575
#6 476.081266889382
#7 476.7400334290244
#10 475.582165863086